# Visión por Computadora 1
## TP 5 
## Alumno: Santiago Fux

In [178]:
#Si queremos que las imágenes sean mostradas en una ventana emergente quitar el inline
%matplotlib inline

# OpenCV-Python utiliza NumPy para el manejo de imágenes
import numpy as np
# cv2 es el módulo python para acceder a OpenCV 
import cv2 as cv
# Usamos las poderosas herramientas de graficación de matplotlib para mostrar imágenes, perfiles, histogramas, etc
import matplotlib.pyplot as plt

from datetime import datetime


* Implementar el detector de fondo naive usando la mediana como estimador. El algoritmo debe recibir el parámetro N (cantidad de frames utilizados para la estimación) y el intervalo de tiempo para recalcular el fondo.
* Se deben generar las mascaras de foreground y aplicarlas a los frames
para segmentar los objetos en movimiento.



In [179]:
def calculate_median(images):
  m = np.median(images, axis = 0)
  return m.astype('uint8')

In [180]:
def get_random_samples(cap, n):
  #obtengo total de frames
  total = cap.get(cv.CAP_PROP_FRAME_COUNT)
  # verifico max
  if n > total:
    n = total
  # obtengo índices random
  idxs = np.random.randint(total, size=(n))
  return idxs
  


In [181]:
def get_mean_vector(cap, idxs):
  res = []
  # save last position
  curr_frame = cap.get(1) # cv.CV_CAP_PROP_POS_FRAMES
  for i in idxs:
    cap.set(1, i) # cv.CV_CAP_PROP_POS_FRAMES
    ret, frame = cap.read()
    res.append(frame)
  # restore frame
  cap.set(1, curr_frame) # cv.CV_CAP_PROP_POS_FRAMES
  res = np.array(res).astype('uint8')
  return res

In [182]:
def get_ts():
  return np.uint32(datetime.timestamp(datetime.now()))

In [183]:
def naive_bg_sub(cap, n, interval, level):
  # obtengo muestras aleatorias
  exit = False
  next_recalc_ts = 0
  frame_rate = cap.get(5) # cv.CV_CAP_PROP_FPS

  naive_process_times = []
  naive_bg_recalculations = []
  while exit == False:
    # obtengo ts actual
    now = get_ts()
    if now > next_recalc_ts:      
      ti = cv.getTickCount()
      # obtengo indices de los frames
      mean_idxs = get_random_samples(cap, n)
      # genero vector con las imagenes(cap) 
      mean_vector = get_mean_vector(cap, mean_idxs)
      # recalculo media de los frames
      mean_img = calculate_median(mean_vector)
      # Aplicar el filtro de mediana
      mean_img = cv.medianBlur(mean_img, 5)
      # reinicio contador
      next_recalc_ts = now + interval
      tf = cv.getTickCount()
      diff_ms = np.uint32((tf-ti) * 1000)/ cv.getTickFrequency()
      naive_bg_recalculations.append(diff_ms)
    
    #obtengo imagen
    success, img = cap.read()

    if success:
      #guardo tiempo inicial
      ti = cv.getTickCount()
      # hago substraccion de fondo conservando el valor absoluto por si el foreground es más oscuro
      diff_img = cv.absdiff(img, mean_img)
      # Extra: Convierto a escala de grises y luego replico la misma para todos los canales
      diff_img = cv.cvtColor(diff_img, cv.COLOR_BGR2GRAY)
      diff_img = cv.merge([diff_img, diff_img, diff_img])
      # binarizo
      _, diff_img = cv.threshold(diff_img, level, 255,cv.THRESH_BINARY)

      # obtengo las mascaras
      fg_masked = cv.bitwise_and(img, cv.bitwise_not(diff_img))
      bg_masked = cv.bitwise_and(mean_img, diff_img)
      # obtengo la imagen final con sólo el background
      bg_img = cv.add(fg_masked, bg_masked)

      # muestro imagen de la máscara
      cv.imshow(f'original', img)
      cv.imshow(f'background only', bg_img)
      cv.imshow(f'mask', diff_img)
      diff_ms = np.uint32((tf-ti) * 1000)/ cv.getTickFrequency()
      # print(f'tiempo demandado:{diff_ms}')
      naive_process_times.append(diff_ms)
    else:
      print(f'Error leyendo video....')
      exit = True

    # verifico si tengo que terminar
    key = cv.waitKey(np.uint32(1000/frame_rate - diff_ms)) & 0xFF
    #cv.destroyAllWindows()
    if key == ord("q"):
      exit = True

  if len(naive_process_times) > 0:
    naive_process_time = np.mean(naive_process_times)
    print(f'naive process time (mean)={naive_process_time}')
    naive_bg_recalculation = np.mean(naive_bg_recalculations)
    print(f'naive recalculation time (mean)={naive_bg_recalculation}')    
  cv.destroyAllWindows()

In [184]:
#procesamos el archivo de gente caminando y pasamos un umbral
n = 10
interval = 5
level = 80
filename = 'vtest.avi'
#inicio captura
capture = cv.VideoCapture(filename)
#disparo procesamiento
naive_bg_sub(capture, n, interval, level)
#detengo captura
capture.release()


naive process time (mean)=2.088825993344538
naive recalculation time (mean)=2.306293195692308


In [185]:
# procesamos video de autos
n = 10
interval = 5
level = 10
filename = 'slow_traffic_small.mp4'
#inicio captura
capture = cv.VideoCapture(filename)
#disparo procesamiento
naive_bg_sub(capture, n, interval, level)
#detengo captura
capture.release()

Error leyendo video....
naive process time (mean)=2.069264603921225
naive recalculation time (mean)=2.7060300297142854


* Comparar con alguno de los métodos vistos en la practica basados en
mezcla de gaussianas

In [186]:

def mog2_bg_sub(cap):
    # elegimos el método MOG2 para hacer substracción
    backSub = cv.createBackgroundSubtractorMOG2()
    # guardamos los valores de procesamiento
    mog2_process_times = []
    # Corremos la sustraccion
    #------------------------
    while True:
        # Leemos un frame
        ret, frame = cap.read()
        if frame is None:
            break
        
        ti = cv.getTickCount()

        # Aplicamos la sustracción al frame leído
        #----------------------------------------
        # Cada frame se utiliza tanto para calcular la máscara de primer plano como para actualizar el fondo.
        # Si se desea cambiar la tasa de aprendizaje utilizada para actualizar el modelo de fondo, es posible
        # establecer una tasa de aprendizaje específica pasando un parámetro al método apply.
        fgMask = backSub.apply(frame)

        tf = cv.getTickCount()
        
        # Escribimos sobre la imagen el número de frame procesado
        cv.rectangle(frame, (10, 2), (100,20), (255,255,255), -1)
        cv.putText(frame, str(cap.get(cv.CAP_PROP_POS_FRAMES)), (15, 15),
                cv.FONT_HERSHEY_SIMPLEX, 0.5 , (0,0,0))
        
        # mostramos frame original e imagen binaria background/foreground
        cv.imshow('original', frame)
        cv.imshow('mask', fgMask)

        diff_ms = np.uint32((tf-ti) * 1000)/ cv.getTickFrequency()
        #print(f'tiempo demandado:{diff_ms}')
        mog2_process_times.append(diff_ms)
        
        # Corresmos hasta que termine o apriete escape
        keyboard = cv.waitKey(30)
        if keyboard == ord("q") or keyboard == 27:
            break

    cv.destroyAllWindows()
    if len(mog2_process_times) > 0:
        mog2_process_time = np.mean(mog2_process_times)
        print(f'mog2 process time (mean)={mog2_process_time}')


In [187]:
# ejectutamos para el video de la gente caminando
filename = 'vtest.avi'
capture = cv.VideoCapture(filename)

#disparamos detección
mog2_bg_sub(capture)
capture.release()



mog2 process time (mean)=1.660285160563522


In [188]:
# ejectutamos para el video de los autos
filename = 'slow_traffic_small.mp4'
capture = cv.VideoCapture(filename)

#disparamos detección
mog2_bg_sub(capture)
capture.release()


mog2 process time (mean)=3.076551454380744


### Tabla comparativa

|Método|Video|Tiempo procesamiento| Tiempo cálculo bg  |
|------|-----|---------|----|
|Naive Bg Sub|vtest.avi|2.088ms|2.306ms|
|Naive Bg Sub|slow_traffic_small.mp4|2.069ms|2.706ms|
|MOG2 Bg Sub|vtest.avi|1,66ms|Incluido en el tiempo de procesamiento|
|MOG2 Bg Sub|slow_traffic_small.mp4|3,07655ms|Incluido en el tiempo de procesamiento|


### Conclusiones

Se verifica según los tiempos relevados que el procesamiento implementado con MOG2 es muy superior en tiempo dado que en el mismo tiempo que insume nuestro algoritmo para el procesamiento de una frame, el MOG2 realiza dicho procesamiento junto con el recálculo del background.  
De todos modos en todos los casos el tiempo insumido está dentro del zócalo de tiempo mínimo necesario entre frames.